# Stablecoin Simulation

---

This notebook is part of a stablecoin project.

---

# Table of Contents

1. [System Requirements](#1.-System-Requirements)
  * [Requirements Analysis](#Requirements-Analysis)
  * [Visual System Mappings](#Visual-System-Mappings)
  * [Mathematical Specification](#Mathematical-Specification)


2. [System Design](#2.-System-Design)
  * [Differential Specification](#Differential-Specification)
  * [Modelling](#Modelling)
  * [Simulation](#Simulation)


3. [System Validation](#3.-System-Validation)


---

# 1. System Requirements

<center><img src="images/requirements.png" alt="Requirements" width="70%"/>

## Requirements Analysis

**Goal:** Simulate a simple collateral-backed stablecoin and examine the effects of external price shocks

**Scope:** Learn how simulations can provide insights into specific dynamics

**Question:** How do price shocks affect loan liquidations?

**Assumptions:**
- The price of the underlying loan asset follows a simple stochastic process
- The agents don't follow a certain strategy. They maintain their loan positions arbitrarily
- Liquidations are triggered immediately and go into a sink

## Visual System Mappings

#### Entity Relationship Diagram

TODO

#### Stock & Flow Diagram

TODO

## Mathematical Specification

TODO

---

# 2. System Design

<center><img src="images/design.png" alt="Design" width="70%"/>

## Differential Specification

TODO

## Modelling

In [ ]:
##############
# 0. IMPORTS #
##############

# Standard libraries
import math
from collections import Counter
import numpy as np
import copy

# utils
import sys
sys.path.append('./utils')
from owner import Owner, OwnerStrategy
from market_stress_simulator import MarketStressSimulator
from metrics import Metrics
from price_simulator import PriceSimulator
from vault import Vault

# Analysis and plotting modules
import pandas as pd
import plotly.express as px

# radCAD modules
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend


In [ ]:
#############
# 0b. UTILS #
#############

initial_coll_price = 2000

owner = Owner(OwnerStrategy.TRADITIONAL)
coll_price_simulator = PriceSimulator()
market_stress_simulator = MarketStressSimulator()
metrics = Metrics(initial_coll_price)


In [ ]:
######################
# 1. STATE VARIABLES #
######################

initial_state = {
    'coll_price': initial_coll_price,
    'pct_coll_price_change': None,
    'negative_shocks': 0,
    'vault': Vault(initial_coll_price, 2000, 1.5, 1000000)
}


########################
# 2. SYSTEM PARAMETERS #
########################

system_params = {}


#######################
# 3. POLICY FUNCTIONS #
#######################

def p_coll_price(params, substep, state_history, previous_state):
    shock_occurence = market_stress_simulator.event_occurrence()
    coll_price_change = coll_price_simulator.calculate_price_change(previous_state['coll_price'], shock_occurence)
    updated_coll_price = previous_state['coll_price'] + coll_price_change
    return {
        'pct_coll_price_change': coll_price_change / previous_state['coll_price'],
        'negative_shock_occurence': shock_occurence and coll_price_change < 0,
        'updated_coll_price': updated_coll_price
    }


#############################
# 4. STATE UPDATE FUNCTIONS #
#############################

def s_coll_price(params, substep, state_history, previous_state, policy_input):
    coll_price = policy_input['updated_coll_price']
    return ('coll_price', coll_price)

def s_pct_coll_price_change(params, substep, state_history, previous_state, policy_input):
    pct_coll_price_change = policy_input['pct_coll_price_change']
    return ('pct_coll_price_change', pct_coll_price_change)

def s_negative_shocks(params, substep, state_history, previous_state, policy_input):
    negative_shocks = previous_state['negative_shocks']
    if policy_input['negative_shock_occurence']:
        negative_shocks += 1
    return ('negative_shocks', negative_shocks)

def s_vault(params, substep, state_history, previous_state, policy_input):
    vault = copy.deepcopy(previous_state['vault'])
    vault.update_coll_price(previous_state['coll_price'])
    metrics.update_indicators(previous_state['coll_price'])
    owner.modify_vault(vault, metrics.get_indicators())
    return ('vault', vault)


##################################
# 5. PARTIAL STATE UPDATE BLOCKS #
##################################

state_update_blocks = [
    {
        'policies': {
            'coll_price': p_coll_price
        },
        'variables': {
            'coll_price': s_coll_price,
            'pct_coll_price_change': s_pct_coll_price_change,
            'negative_shocks': s_negative_shocks,
        }
    },
    {
        'policies': {},
        'variables': {
            'vault': s_vault
        }
    }
]

## Simulation

In [ ]:
####################
# 6. CONFIGURATION #
####################

model = Model(
    initial_state=initial_state,
    state_update_blocks=state_update_blocks,
    # params=system_params
)

simulation = Simulation(
    model=model,
    timesteps=300,  # Number of timesteps
    runs=10  # Number of Monte Carlo Runs
)

experiment = Experiment([simulation])
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS, drop_substeps=True)


################
# 7. EXECUTION #
################

raw_result = experiment.run()


####################################
# 8. SIMULATION OUTPUT PREPARATION #
####################################

simulation_result = pd.DataFrame(raw_result)

simulation_result['coll_balance'] = [t.coll_balance for t in simulation_result['vault']]
simulation_result['coll_value'] = [t.coll_value for t in simulation_result['vault']]
simulation_result['loan_value'] = [t.loan_value for t in simulation_result['vault']]
simulation_result['liquidated'] = [t.liquidated for t in simulation_result['vault']]
simulation_result['liquidated_value'] = [t.liquidated_value for t in simulation_result['vault']]
simulation_result['liquidation_buffer'] = [t.liquidation_buffer for t in simulation_result['vault']]
simulation_result['volatility_buffer'] = [t.volatility_buffer for t in simulation_result['vault']]

simulation_result.drop(columns=['vault'], inplace=True)

simulation_result.head(10)


In [ ]:
########################################
# 9. SIMULATION ANALYSIS: SHOCK EVENTS #
########################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['negative_shocks'],
    title="Negative Market Shocks",
    labels={"value": "event number"},
    color='run'
)

In [ ]:
###################################################
# 9. SIMULATION ANALYSIS: COLLATERAL PRICE CHANGE #
###################################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['pct_coll_price_change'],
    title="Percentage of Collateral Price Change",
    color='run'
)

In [ ]:
############################################
# 9. SIMULATION ANALYSIS: COLLATERAL PRICE #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['coll_price'],
    title="Collateral Price",
    color='run'
)

In [ ]:
############################################
# 9. SIMULATION ANALYSIS: COLLATERAL VALUE #
############################################

pd.options.plotting.backend = "plotly"
simulation_result.plot(
    kind='line',
    x='timestep',
    y=['coll_value'],
    title="Vault",
    color='run'
)

In [ ]:
##################################
# 9. SIMULATION ANALYSIS: VAULTS #
##################################

px.bar(
    simulation_result,
    x='run', # Variable on the horizontal axis
    y=['loan_value', 'liquidation_buffer', 'volatility_buffer', 'liquidated_value'], # Variables on the vertical axis
    # range_y=[0, max_coll_value + 100], # Set the range for the y-axis values
    title="Vault",
    animation_frame='timestep', # Set the variable to animate
)

---

# 3. System Validation

<center><img src="images/validation.png" alt="Validation" width="70%"/>

In [ ]:
pd.options.plotting.backend = "plotly"

test_metrics = Metrics(2000)
simulation_result_subset = simulation_result[['coll_price', 'run']].drop([0]) # first row already passed via Matrics constructor
single_run = simulation_result_subset[simulation_result_subset.run == 6]

for p in single_run['coll_price']:
    test_metrics.update_indicators(p)
    
test_indicators = test_metrics.get_all_indicators()

test_indicators.plot(
    kind='line',
    y=['coll_price', 'moving_average', 'exponential_moving_average']
)

In [ ]:
test_indicators.head(301)